# Unidad 1 - Extracción y Procesamiento de Texto

Para todos los ejercicios de web scraping contemple usar un temporizador para no
matar los sitios con las consultas.

## Librerías

Se utilizarán las siguientes librerías y paquete a lo largo de los siguientes ejercicios:

In [ ]:
!apt-get update -qq
!apt-get install -qq poppler-utils
!apt-get install -qq tesseract-ocr

!pip install PyPDF2
!pip install pdf2image
!pip install Pillow
!pip install pytesseract

!pip install easyocr
!pip install python-docx

!pip install SpeechRecognition

!pip install beautifulsoup4
!pip install requests

!pip install autocorrect
!pip install pyspellchecker
!pip install textblob

!which pdfinfo
!which tesseract

# Configurar en español
!wget https://github.com/tesseract-ocr/tessdata/raw/main/spa.traineddata
!mv spa.traineddata /usr/local/share/tessdata/

In [38]:
import os
from google.colab import drive

from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import easyocr
from docx import Document

import speech_recognition as sr

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

from textblob import TextBlob
from autocorrect import Speller
from spellchecker import SpellChecker

from IPython.display import display, Markdown

import PyPDF2

## Ruta de acceso principal

Se utilizará la siguiente ruta para acceso a los archivos del trabajo:

In [39]:
drive.mount('/content/drive')
ruta='/content/drive/MyDrive/TUIA/IA4.2 NLP/Unidad 1/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Ejercicio 0
En el campus se encuentran dos archivos en la carpeta “Ejemplos
codificación”. Investíguelos con las herramientas vistas en clase. Puede usar
también la herramienta hexdump para verlos por dentro.

In [40]:
file_path = ruta+'EJEMPLO DE UTF-8.txt'

# Abre el archivo en modo de lectura ('r')
with open(file_path, 'r', encoding='utf-8') as archivo:
    # Lee todo el contenido del archivo
    contenido = archivo.read()

# Imprime el contenido del archivo
print(contenido)

Cómo estas



In [ ]:
file_path = ruta+'EJEMPLO DE 8859-15.txt'

# Abre el archivo en modo de lectura ('r')
with open(file_path, 'r', encoding='iso-8859-1') as archivo:
    # Lee todo el contenido del archivo
    contenido = archivo.read()

# Imprime el contenido del archivo
print(contenido)

Cómo estás



Pruebe intencionalmente abrir el archivo con la codificación equivocada (abra el
UTF-8 como 8859-15) y compare las salidas.

In [ ]:
file_path = ruta+'EJEMPLO DE UTF-8.txt'

# Abre el archivo en modo de lectura ('r')
with open(file_path, 'r', encoding='iso-8859-1') as archivo:
    # Lee todo el contenido del archivo
    contenido = archivo.read()

# Imprime el contenido del archivo
print(contenido)

CÃ³mo estas



## Ejercicio 1
Con los archivos pedidos en diferentes formatos (pdf, jpg, ...otros) la
clase anterior, extraiga la información que considere relevante de los mismos utilizando las librerías desarrolladas en clase.

### i.
Obtenga texto de un libro escaneado en pdf (un pdf que no tenga el texto
codificado como tal, es decir, que sea necesario usar ocr). Pruebe usando
pytesseract en ingles (por defecto) y luego configurándolo en español.

In [ ]:
# Ruta al archivo PDF
archivo_pdf = ruta+'ej 1-i.pdf'

# Convierte el PDF en una lista de imágenes
imagenes = convert_from_path(archivo_pdf)

# Guarda las imágenes
for i, imagen in enumerate(imagenes):
    imagen.save('pagina{}.png'.format(i), 'PNG')

# Ruta a la imagen
archivo_imagen = 'pagina0.png'

# Abre la imagen
imagen = Image.open(archivo_imagen)

# Usa pytesseract para convertir la imagen en texto
texto = pytesseract.image_to_string(imagen)

# Imprime el texto extraído
print(texto)

In [ ]:
#VER ERROR
texto = pytesseract.image_to_string(imagen, lang='spa')
print(texto)

### ii.
Obtenga texto de una imagen (png, bmp).

In [ ]:
# Crea un lector OCR en español
lector = easyocr.Reader(['es'])

# Ruta a la imagen
archivo_imagen = ruta+'api.bmp'

# Usa EasyOCR para convertir la imagen en texto
resultado = lector.readtext(archivo_imagen)

# El resultado es una lista de tuplas, donde cada tupla representa un bloque de texto.
# La primera posición de la tupla es la ubicación del bloque de texto en la imagen,
# y la segunda posición es el texto en sí.

# Imprime el texto extraído
for ubicacion, texto, _ in resultado:
    print(f"Texto: {texto}, Ubicación: {ubicacion}")

### iii.
Obtenga texto de un archivo word.

In [ ]:
# Abre el archivo DOCX
doc = Document(ruta+'prueba.docx')

# Extrae el texto de cada párrafo en el documento
for parrafo in doc.paragraphs:
    print(parrafo.text)

Si estás leyendo esto es porque todo funciona muy bien.


### iv.
Obtenga texto de un archivo de audio.

In [ ]:
# Crea un objeto Recognizer
r = sr.Recognizer()

# Abre el archivo de audio
with sr.AudioFile(ruta+'audio1.wav') as source:
    # Lee el archivo de audio
    audio_data = r.record(source)
    # Transcribe el audio a texto
    texto = r.recognize_google(audio_data, language="es-ES")
    print(texto)

qué tal Juli dale bárbaro eh yo probablemente hoy te estoy subiendo todas las barandas y algunas pasarelas no sé cuál es pero no voy a llegar al total total de la info


## Ejercicio 2
El Ministerio de Turismo y Deportes de la Nación permite explorar
tableros de información en línea tableros.yvera.tur.ar. Explore la página y utilizando una librería de web scraping extraiga los valores del tablero de indicadores de
Objetivos de Desarrollo Sostenible en una tabla y el texto limpio de la metodología
de los mismos.

In [42]:
url = "https://tableros.yvera.tur.ar/tablero_ODS/"

response = requests.get(url, verify=False)
response.encoding = 'utf-8'  # Establece la codificación a utf-8
soup = BeautifulSoup(response.text, 'html.parser') # Desactivar verificación SSL

# Encuentra todos los elementos <div> con la clase "jsx-3759278058 f4"
data = soup.find_all('div', class_='inner')

data_list = []
for div in data:
    valor_text = div.text.strip().split('\n\n\n')[0]
    valor = re.sub(r'[^\d,]', '', valor_text)  # Extrae solo los números y puntos decimales
    descripcion = div.text.strip().split('\n\n\n')[1] if len(div.text.strip().split('\n\n\n')) > 1 else None
    div_data = {
        'valor': valor,
        'descripcion': descripcion
    }
    data_list.append(div_data)

df = pd.DataFrame(data_list)
df

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'tableros.yvera.tur.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,valor,descripcion
0,"1,2",Puestos de trabajo en industrias turísticas (2...
1,"44,43",Tasa de informalidad del empleo asalariado tur...
2,"1,68",PIB turístico respecto del PIB total (2022)
3,"33,2",Propensión a viajar (2022)
4,"34,1",Índice de Gini de viajes turísticos (2022)
5,"36,4",Índice de Gini de viajes turísticos de ocio (2...
6,"44,1",Índice de Gini del gasto turístico (2022)


## Ejercicio 3
Investigue cómo extraer información de vuelo en la página
flybondi.com, haga un programa en python para verificar si bajó el precio de un
vuelo determinado.

In [43]:
url = "https://flybondi.com/ar/search/results?adults=2&children=0&currency=ARS&departureDate=2024-04-27&fromCityCode=BUE&infants=0&returnDate=2024-05-01&toCityCode=BRC&utm_origin=search_bar/"

response = requests.get(url)
response.encoding = 'utf-8'  # Establece la codificación a utf-8
soup = BeautifulSoup(response.text, 'html.parser')

# Encuentra todos los elementos <div> con la clase "jsx-3759278058 f4"
price_elements = soup.find_all('div', class_='ph3 ph0-l')


# Extrae el texto de cada elemento de precio y lo imprime
for price_element in price_elements:
    price = price_element.text  # Elimina espacios en blanco al principio y al final
    print(price)



## Ejercicio 4
Con el objetivo de estudiar la serie de tipo de cambio oficial y su
volatilidad de forma visual. Utilice una librería de web scraping y elija una dirección
que posea esta información en una frecuencia diaria y recolecte esa información
(por ejemplo dolarhoy.com). Documente el proceso debidamente.
Opcional: Una vez obtenidos los datos requeridos realice una gráfica que muestre la
serie y calcule los valores promedio mensual de la serie. Su máximo valor y su
mínimo. Presente los datos por pantalla en forma ordenada.

## Ejercicio 5
Obtener los libros en formato pdf de la página del autor argentino
Hernán Casciari de la editorial Orsai, estos textos son de libre utilización. Pasar a un
archivo pkl el texto de los libros en pdf.

## Ejercicio 6
Realizar en Colab un script Python que haga los siguientes pasos:

### i.
Obtener el HTML de la página https://es.wikipedia.org/wiki/Argentina

### ii.
Con BeautifulSoup, Obtener el texto contenido en el <div> cuyo id='mw-
content-text’

### iii.
Aplicar RecursiveCharacterTextSplitter con los parámetros chunk_size = 300 en
el texto de wikipedia.

### iv.
Aplicar RecursiveCharacterTextSplitter con los parámetros chunk_size = 300,
separators = ["\n\n", "\n"] en el texto de wikipedia.

### v.
Aplicar CharacterTextSplitter() en el texto de wikipedia.

### vi.
En cada caso, graficar el histograma de como se distribuyen los tamaños de
los segmentos (chunks) y sacar conclusiones.

## Ejercicio 7
En la siguiente carpeta,
https://drive.google.com/drive/folders/1iCiOQ8P8CHFFLKiWc2li9-x3Ud-Tl0po?usp=drive_link
podrá descargar recursos que presentan algunos desafios para la extracción de
texto. Practique los códigos y librerías propuestos en la Unidad 1 y compare los
resultados con sus compañer@s.

###i

In [44]:
# Ruta al archivo PDF
archivo_pdf = ruta+'(INTRODUCTION TO THE THEORY OF STATISTICS) Alexander, Franklin Graybill and Duane Boes Mood - Introduccion a la teoria de la Estadistica-Aguilar (1969).pdf'


# Convierte el PDF en una lista de imágenes
imagenes = convert_from_path(archivo_pdf)

# Guarda las imágenes
for i, imagen in enumerate(imagenes[:16]):
    imagen.save('pagina{}.png'.format(i), 'PNG')

# Ruta a la imagen
archivo_imagen = 'pagina15.png'

# Abre la imagen
imagen = Image.open(archivo_imagen)

# Usa pytesseract para convertir la imagen en texto
texto = pytesseract.image_to_string(imagen)

# Configurar el corrector ortográfico para español
spell = Speller(lang="es")
texto = re.sub(r'-+\n', '', texto) #elimina guiones y saltos de línea
texto = re.sub(r'\n', ' ', texto) #elimina saltos de línea por espacios
texto_corregido=spell(texto)

# Mostrar el texto en un formato más legible
display(Markdown(texto_corregido))

capitulo 1 INTRODUCCION  1-1. Estadistica.—Para situar este libro en la perspectiva adecuada es necesario que empecemos por considerar qué es la estadistica. La concepción profana de estadistica suele incluir la recogida de grandes masas de datos y la presentación de estos en tablas 0 graficos; puede incluir también el cálculo de totales, promedios, porcentajes, etc. En todo caso, estas operaciones, més 0 menos rutinarias, son una parte, pero solo una parte incidental de la estadistica. Estadistica es también el diseño de experimentos, el diseño de sobrevisiones muestras, la reducción y el proceso de datos, y otras muchas cuestiones.  Describiremos la estadistica como la tecnologia del método cientifico. La estadistica proporciona instrumentos para la toma de decisiones cuando prevalecen condiciones de incertidumbre. Estos instrumentos pueden ser de aplicación y utilidad completamente general en cualquier campo de la ciencia: físico, biológico, social, etcétera. Son aplicables no solo en el mundo cientifico, sino también en el de la empresa y en el de los asuntos cotidianos. Por otra parte, ciertos instrumentos pueden estar especialmente diseñados para campos especiales de la investigación.  La estadistica puede dividirse en dos amplias ramas: 1) estadistica descriptiva, que est4 relacionada con el resumen de datos y la descripción. de estos; 2) estadistica inferencial, relacionada con el proceso de utilizar datos para tomar decisiones en el caso mds general del gue forman parte estos datos. El proceso de tomar decisiones en situaciones generales, sobre la base de una información incompleta contenida en datos muestras, es arriesgado y no puede realizarse con certeza; la probabilidad es una medida de esta incertidumbre. Hay dos tipos de incertidumbre con los que tenemos que enfrentarnos: 1) la incertidumbre debida a la aleatoriedad, y 2) la incertidumbre debida a nuestra ignorancia del verdadero estado del sistema. Lo acabaremos con un ejemplo.  La compania A cultiva cierta clase de plantas, recoleccion las semillas y las envase en paquetes de 25 semillas cada uno. Un almacén de venta al por menor adquiere algunos de los paquetes y garantiza a sus compradores que 22 al menos de las 25 semillas de cada paquete crecerán; en caso contrario, les dar4 otro paquete libre de todo gasto. El almacenista tiene dos tipos de incertidumbre  3 

###ii

In [47]:
# Ruta al archivo PDF
libro = ruta+'el_principito_saint_exupery.pdf'

# Abre el archivo en modo binario de lectura ('rb')
with open(libro, 'rb') as archivo:
    # Crea un objeto PdfFileReader
    lector = PyPDF2.PdfReader (archivo)

    # Inicializa una cadena vacía para almacenar el texto
    texto = ''

    # Itera sobre todas las páginas del PDF
    for i in range(len(lector.pages)):
        # Obtiene la página
        pagina = lector.pages[i]

        # Extrae el texto de la página y lo añade a la cadena de texto
        texto += pagina.extract_text()

# Configurar el corrector ortográfico para español
spell = Speller(lang="es")
texto = re.sub(r'-+\n', '', texto) #elimina guiones y saltos de línea
texto = re.sub(r'\n', ' ', texto) #elimina saltos de línea por espacios

# Mostrar el texto en un formato más legible
display(Markdown(texto_corregido))

capitulo 1 INTRODUCCION  1-1. Estadistica.—Para situar este libro en la perspectiva adecuada es necesario que empecemos por considerar qué es la estadistica. La concepción profana de estadistica suele incluir la recogida de grandes masas de datos y la presentación de estos en tablas 0 graficos; puede incluir también el cálculo de totales, promedios, porcentajes, etc. En todo caso, estas operaciones, més 0 menos rutinarias, son una parte, pero solo una parte incidental de la estadistica. Estadistica es también el diseño de experimentos, el diseño de sobrevisiones muestras, la reducción y el proceso de datos, y otras muchas cuestiones.  Describiremos la estadistica como la tecnologia del método cientifico. La estadistica proporciona instrumentos para la toma de decisiones cuando prevalecen condiciones de incertidumbre. Estos instrumentos pueden ser de aplicación y utilidad completamente general en cualquier campo de la ciencia: físico, biológico, social, etcétera. Son aplicables no solo en el mundo cientifico, sino también en el de la empresa y en el de los asuntos cotidianos. Por otra parte, ciertos instrumentos pueden estar especialmente diseñados para campos especiales de la investigación.  La estadistica puede dividirse en dos amplias ramas: 1) estadistica descriptiva, que est4 relacionada con el resumen de datos y la descripción. de estos; 2) estadistica inferencial, relacionada con el proceso de utilizar datos para tomar decisiones en el caso mds general del gue forman parte estos datos. El proceso de tomar decisiones en situaciones generales, sobre la base de una información incompleta contenida en datos muestras, es arriesgado y no puede realizarse con certeza; la probabilidad es una medida de esta incertidumbre. Hay dos tipos de incertidumbre con los que tenemos que enfrentarnos: 1) la incertidumbre debida a la aleatoriedad, y 2) la incertidumbre debida a nuestra ignorancia del verdadero estado del sistema. Lo acabaremos con un ejemplo.  La compania A cultiva cierta clase de plantas, recoleccion las semillas y las envase en paquetes de 25 semillas cada uno. Un almacén de venta al por menor adquiere algunos de los paquetes y garantiza a sus compradores que 22 al menos de las 25 semillas de cada paquete crecerán; en caso contrario, les dar4 otro paquete libre de todo gasto. El almacenista tiene dos tipos de incertidumbre  3 

###iii

In [ ]:
file_path = ruta+'125cuentos.txt'

# Abre el archivo en modo de lectura ('r')
with open(file_path, 'r', encoding='utf-8') as archivo:
    # Lee todo el contenido del archivo
    texto = archivo.read()

texto = re.sub(r' -+\n', '', texto) #elimina guiones y saltos de línea
texto = re.sub(r'\n', ' ', texto) #elimina saltos de línea por espacios
# Imprime el contenido del archivo

# Mostrar el texto en un formato más legible
display(Markdown(texto))